<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-20 12:06:48
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.33 C
CY Investment:  1.60 C
Reserve:  2.44 L
Current:  1.41 C
-------------------
Today PnL: 41.86 K (0.3%)
Current PnL: -33.11 L (-20.69%)
CY Booked + Current PnL: -17.43 L (-10.89%)
-------------------
Total profit:  1.30 L
Total loss:  -34.41 L
-------------------
Total Booked + Current PnL: 9.55 L (7.18%)
Total Booked PnL: 42.66 L (32.07%)
Curr Year Booked PnL: 15.68 L (11.09%)
Prev Year Booked PnL: 26.98 L (20.29%)
Est FTT:  2.41 C
Est FTT PnL: 99.72 L (70.56%)
Deployed:  1.33 C
Current:  1.41 C
CAGR/XIRR %: 3.14%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,0.33,1.61,19.64,21.57,19553.0,1575.0,99555.0,397.04,-12.38,55.0,H-SC,5.48,167.0,0.08,0.72,45.28,XR,ATH,FINANCE
18,COALINDIA,1.38,3.75,13.42,17.67,20949.0,5642.0,156103.0,478.51,24.82,53.0,L-LC,11.23,185.0,0.27,1.12,25.31,XY25,ATH,MINING
77,TTKPRESTIG,-0.34,-28.45,39.90,0.09,28769.0,-28675.0,72102.0,770.00,67.34,36.0,M-SC,9.70,250.0,-1.00,0.52,0.00,OX40N,NTT,DURABLES
36,ICICIGI,0.03,2.58,16.33,19.34,33522.0,5171.0,205280.0,2252.93,-13.70,59.0,X-MC,2.31,66.0,0.15,1.48,18.93,X40,ATH,INSURANCE
51,MEDANTA,-1.55,-6.32,32.44,24.08,38582.0,-8018.0,118932.0,1486.00,-6.94,47.0,X-SC,10.35,87.0,-0.21,0.86,8.93,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,8.28,13.62,28.11,45.57,83561.0,35645.0,297264.0,7934.0,-26.98,72.0,H-MC,5.98,125.0,0.43,2.14,32.00,AR,NTT,ELECTRICAL
0,5PAISA,6.76,-36.29,77.20,12.90,107473.0,-79282.0,139214.0,593.0,105.53,49.0,H-SC,7.04,170.0,-0.74,1.00,15.76,OX40N,NTT,FINANCE
65,SIEMENS,5.21,-12.17,42.90,25.51,70122.0,-22640.0,163455.0,4671.5,37.32,60.0,H-LC,2.00,51.0,-0.32,1.18,20.45,AR,ATH,ELECTRICAL
43,IRCTC,2.99,-22.35,96.96,52.93,203657.0,-60466.0,210042.0,1269.0,-42.52,59.0,H-MC,12.28,119.0,-0.30,1.51,6.65,XY24,BTT,TRAVEL
82,VOLTAS,1.97,20.67,20.32,45.20,47015.0,39633.0,231375.0,1856.0,38.08,65.0,X-MC,6.01,79.0,0.84,1.67,28.87,XY25,NTT,AC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,EASEMYTRIP,-5.31,-51.82,202.41,45.70,172427.0,-91628.0,85187.0,26.40,-11.50,65.0,M-SC,20.13,204.0,-0.53,0.61,40.35,XY24,NTT,TRAVEL
7,ATULAUTO,-3.63,-17.73,76.94,45.57,124781.0,-34952.0,162180.0,844.00,3510.81,53.0,M-SC,3.94,248.0,-0.28,1.17,22.73,XY24,NTT,AUTO
30,HAPPSTMNDS,-2.33,-52.83,291.88,84.85,176459.0,-67709.0,60456.0,1480.71,-43.83,35.0,H-SC,25.89,152.0,-0.38,0.44,1.60,AR,ATH,IT
14,BSOFT,-2.16,-31.54,120.26,50.79,115796.0,-44365.0,96288.0,831.70,-12.25,38.0,H-SC,6.55,172.0,-0.38,0.69,12.11,XR,ATH,IT
55,RAJESHEXPO,-2.15,-69.31,226.09,0.07,95173.0,-95082.0,42095.0,518.00,1442.93,38.0,L-SC,9.00,268.0,-1.00,0.30,4.69,OX40N,NTT,JEWELLERY


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,-0.34,1.20,105.45,107.91,157556.0,1769.0,149413.0,1641.55,-9.18,57.0,M-SC,8.29,220.0,0.01,1.08,12.41,OX40N,ATH,CABLES
77,TTKPRESTIG,-0.34,-28.45,39.90,0.09,28769.0,-28675.0,72102.0,770.00,67.34,36.0,M-SC,9.70,250.0,-1.00,0.52,0.00,OX40N,NTT,DURABLES
72,TATAELXSI,0.28,-30.06,88.77,32.03,81852.0,-39628.0,92207.0,9161.00,-12.47,35.0,H-SC,8.09,158.0,-0.48,0.66,3.28,OX40N,NTT,IT
66,SIS,-1.86,-31.99,78.29,21.26,59123.0,-35514.0,75518.0,528.00,1771.50,29.0,H-SC,4.49,168.0,-0.60,0.54,4.11,OX40N,NTT,MISC
40,INDIGOPNTS,-2.11,-32.86,48.98,0.02,57401.0,-57367.0,117192.0,1408.00,78.57,26.0,M-SC,9.92,222.0,-1.00,0.84,1.94,OX40N,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VOLTAS,1.97,20.67,20.32,45.2,47015.0,39633.0,231375.0,1856.0,38.08,65.0,X-MC,6.01,79.0,0.84,1.67,28.87,XY25,NTT,AC


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,1.38,3.75,13.42,17.67,20949.0,5642.0,156103.0,478.51,24.82,53.0,L-LC,11.23,185.0,0.27,1.12,25.31,XY25,ATH,MINING
26,FINCABLES,-0.34,1.20,105.45,107.91,157556.0,1769.0,149413.0,1641.55,-9.18,57.0,M-SC,8.29,220.0,0.01,1.08,12.41,OX40N,ATH,CABLES
50,MASFIN,0.33,1.61,19.64,21.57,19553.0,1575.0,99555.0,397.04,-12.38,55.0,H-SC,5.48,167.0,0.08,0.72,45.28,XR,ATH,FINANCE
1,ABB,8.28,13.62,28.11,45.57,83561.0,35645.0,297264.0,7934.00,-26.98,72.0,H-MC,5.98,125.0,0.43,2.14,32.00,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,8.28,13.62,28.11,45.57,83561.0,35645.0,297264.0,7934.00,-26.98,72.0,H-MC,5.98,125.0,0.43,2.14,32.00,AR,NTT,ELECTRICAL
50,MASFIN,0.33,1.61,19.64,21.57,19553.0,1575.0,99555.0,397.04,-12.38,55.0,H-SC,5.48,167.0,0.08,0.72,45.28,XR,ATH,FINANCE
26,FINCABLES,-0.34,1.20,105.45,107.91,157556.0,1769.0,149413.0,1641.55,-9.18,57.0,M-SC,8.29,220.0,0.01,1.08,12.41,OX40N,ATH,CABLES
39,INDIAMART,0.73,-6.48,121.05,106.72,139624.0,-7992.0,115344.0,4810.62,-57.81,44.0,H-SC,7.93,140.0,-0.06,0.83,15.38,AR,ATH,MISC
85,ZYDUSLIFE,-0.24,-5.07,42.88,35.63,85307.0,-10630.0,198944.0,1286.17,-17.57,49.0,H-MC,5.20,120.0,-0.12,1.43,10.75,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,INFY,-1.13,-7.76,45.54,34.24,131435.0,-24273.0,288615.0,1972.00,-27.43,23.0,X-LC,4.64,3.0,-0.18,2.08,0.00,X40,NTT,IT
8,AWL,-0.71,-35.71,144.50,57.19,302969.0,-116460.0,209667.0,485.00,-63.27,25.0,X-SC,17.65,81.0,-0.38,1.51,0.00,XY24,NTT,FMCG
33,HCLTECH,-0.56,-6.40,32.30,23.83,73140.0,-15483.0,226441.0,1908.19,-1.57,25.0,X-LC,4.09,8.0,-0.21,1.63,8.64,X40,ATH,IT
73,TCS,0.22,-23.08,60.65,23.58,201838.0,-99838.0,332791.0,4311.59,-33.14,27.0,X-LC,5.49,2.0,-0.49,2.40,0.22,X40,ATH,IT
13,BERGEPAINT,-0.21,-18.68,48.96,21.14,90518.0,-42457.0,184882.0,680.00,-20.45,28.0,X-MC,13.28,68.0,-0.47,1.33,0.00,XY24,NTT,PAINTS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,-0.77,1.84,18.59,20.77,44327.0,4300.0,238444.0,1644.00,-9.71,51.0,X-MC,0.90,64.0,0.10,1.72,9.37,X40N,NTT,BREWERIES
21,DABUR,1.61,-0.34,44.19,43.70,109250.0,-839.0,247229.0,735.00,-9.49,49.0,X-MC,1.39,70.0,-0.01,1.78,14.66,XY24,BTT,FMCG
65,SIEMENS,5.21,-12.17,42.90,25.51,70122.0,-22640.0,163455.0,4671.50,37.32,60.0,H-LC,2.00,51.0,-0.32,1.18,20.45,AR,ATH,ELECTRICAL
36,ICICIGI,0.03,2.58,16.33,19.34,33522.0,5171.0,205280.0,2252.93,-13.70,59.0,X-MC,2.31,66.0,0.15,1.48,18.93,X40,ATH,INSURANCE
34,HINDUNILVR,1.88,-9.67,25.83,13.67,59980.0,-24853.0,232210.0,2922.00,-32.95,45.0,X-LC,2.86,9.0,-0.41,1.67,9.62,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,-0.79,-47.17,155.92,35.20,105990.0,-60693.0,67977.0,2096.00,-10.90,33.0,X-SC,21.74,91.0,-0.57,0.49,0.0,X40,NTT,FOOTWEAR
8,AWL,-0.71,-35.71,144.50,57.19,302969.0,-116460.0,209667.0,485.00,-63.27,25.0,X-SC,17.65,81.0,-0.38,1.51,0.0,XY24,NTT,FMCG
13,BERGEPAINT,-0.21,-18.68,48.96,21.14,90518.0,-42457.0,184882.0,680.00,-20.45,28.0,X-MC,13.28,68.0,-0.47,1.33,0.0,XY24,NTT,PAINTS
42,INFY,-1.13,-7.76,45.54,34.24,131435.0,-24273.0,288615.0,1972.00,-27.43,23.0,X-LC,4.64,3.0,-0.18,2.08,0.0,X40,NTT,IT
53,PGHH,-0.42,-14.05,53.59,32.01,98348.0,-30000.0,183520.0,17616.87,-41.89,31.0,X-MC,9.46,57.0,-0.31,1.32,0.0,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,-1.09,-31.46,108.42,42.84,48304.0,-20453.0,44553.0,424.00,-55.37,46.0,X-SC,18.18,80.0,-0.42,0.32,7.17,XY24,NTT,MISC
12,BATAINDIA,-0.79,-47.17,155.92,35.20,105990.0,-60693.0,67977.0,2096.00,-10.90,33.0,X-SC,21.74,91.0,-0.57,0.49,0.00,X40,NTT,FOOTWEAR
58,RELAXO,-1.16,-53.23,225.67,52.31,153201.0,-77273.0,67887.0,1176.00,-50.25,41.0,X-SC,14.02,92.0,-0.50,0.49,1.78,X40N,NTT,FOOTWEAR
51,MEDANTA,-1.55,-6.32,32.44,24.08,38582.0,-8018.0,118932.0,1486.00,-6.94,47.0,X-SC,10.35,87.0,-0.21,0.86,8.93,XY24,NTT,HEALTHCARE
35,HONAUT,-0.37,-23.90,88.34,43.32,109489.0,-38932.0,123940.0,58357.33,-33.23,36.0,X-SC,11.59,90.0,-0.36,0.89,0.63,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,0.23,-13.93,38.33,19.06,112218.0,-47398.0,292768.0,452.00,-52.69,50.0,X-LC,18.23,1.0,-0.42,2.11,6.01,X40,NTT,FMCG
73,TCS,0.22,-23.08,60.65,23.58,201838.0,-99838.0,332791.0,4311.59,-33.14,27.0,X-LC,5.49,2.0,-0.49,2.40,0.22,X40,ATH,IT
42,INFY,-1.13,-7.76,45.54,34.24,131435.0,-24273.0,288615.0,1972.00,-27.43,23.0,X-LC,4.64,3.0,-0.18,2.08,0.00,X40,NTT,IT
75,TMPV,0.40,-16.18,59.07,33.33,161316.0,-52707.0,273093.0,600.00,-82.53,57.0,X-LC,2.89,4.0,-0.33,1.97,11.65,XY24,NTT,AUTO
33,HCLTECH,-0.56,-6.40,32.30,23.83,73140.0,-15483.0,226441.0,1908.19,-1.57,25.0,X-LC,4.09,8.0,-0.21,1.63,8.64,X40,ATH,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,0.35,-31.30,93.42,32.89,49554.0,-24162.0,53044.0,1800.00,-350.00,57.0,L-MC,11.29,258.0,-0.49,0.38,46.08,XR,NTT,BANKS
67,SONACOMS,1.34,-7.42,50.24,39.09,47050.0,-7508.0,93651.0,804.02,-26.68,64.0,M-MC,8.30,193.0,-0.16,0.67,32.49,AR,ATH,AUTO
50,MASFIN,0.33,1.61,19.64,21.57,19553.0,1575.0,99555.0,397.04,-12.38,55.0,H-SC,5.48,167.0,0.08,0.72,45.28,XR,ATH,FINANCE
18,COALINDIA,1.38,3.75,13.42,17.67,20949.0,5642.0,156103.0,478.51,24.82,53.0,L-LC,11.23,185.0,0.27,1.12,25.31,XY25,ATH,MINING
82,VOLTAS,1.97,20.67,20.32,45.20,47015.0,39633.0,231375.0,1856.00,38.08,65.0,X-MC,6.01,79.0,0.84,1.67,28.87,XY25,NTT,AC


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,8.28,13.62,28.11,45.57,83561.0,35645.0,297264.0,7934.00,-26.98,72.0,H-MC,5.98,125.0,0.43,2.14,32.00,AR,NTT,ELECTRICAL
25,EASEMYTRIP,-5.31,-51.82,202.41,45.70,172427.0,-91628.0,85187.0,26.40,-11.50,65.0,M-SC,20.13,204.0,-0.53,0.61,40.35,XY24,NTT,TRAVEL
11,BANDHANBNK,1.87,-16.95,131.70,92.42,304672.0,-47221.0,231338.0,400.00,98.28,70.0,H-SC,6.29,174.0,-0.15,1.67,28.73,XY24,NTT,BANKS
67,SONACOMS,1.34,-7.42,50.24,39.09,47050.0,-7508.0,93651.0,804.02,-26.68,64.0,M-MC,8.30,193.0,-0.16,0.67,32.49,AR,ATH,AUTO
83,WHIRLPOOL,1.94,-25.36,143.75,81.92,123165.0,-29118.0,85680.0,2270.00,-56.34,63.0,M-SC,27.57,210.0,-0.24,0.62,20.35,XR,NTT,DURABLES


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.79
1,25,45.52
2,50,77.39


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.12
MC    30.12
LC    24.80
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.93
X40      23.95
X40N     14.83
AR        9.05
XY25      8.77
XR        8.54
OX40N     7.12
SR        0.85
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    22.59
H-SC    22.18
X-LC    21.44
X-SC    11.55
M-SC    10.65
H-MC     5.08
M-MC     2.07
H-LC     1.18
L-LC     1.12
M-LC     1.06
L-SC     0.74
L-MC     0.38
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.56,-14.24,52.87
FINANCE,13.14,-14.49,59.54
IT,11.63,-35.82,103.88
MISC,6.88,-35.88,91.46
ELECTRICAL,6.33,-6.72,45.66
PAINTS,5.00,-32.81,52.54
INSURANCE,4.80,-1.83,36.67
PHARMA,4.07,-3.87,36.57
AUTO,3.81,-17.99,62.99


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3391735.0,21
X40,1395370.0,15
AR,1314315.0,10
XR,1306644.0,12
X40N,1061728.0,10
OX40N,801023.0,10
XY25,399131.0,6
SR,302010.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3714173.0,24
M-SC,1441005.0,14
X-MC,1376899.0,15
X-LC,1296096.0,12
X-SC,1200339.0,10
H-MC,372525.0,3
L-SC,187623.0,2
M-LC,145298.0,1
M-MC,97373.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1267450.0      6
           AR         953732.0      5
           XR         841074.0      7
M-SC       XY24       832251.0      6
X-LC       X40        665832.0      6
X-SC       XY24       618249.0      4
X-MC       X40        548246.0      7
           X40N       429427.0      4
X-SC       X40N       400798.0      4
M-SC       OX40N      355943.0      5
H-SC       OX40N      349907.0      4
           SR         302010.0      2
X-LC       XY24       270360.0      2
           X40N       231503.0      2
H-MC       XY24       203657.0      1
X-MC       XY24       199768.0      2
           XY25       199458.0      2
X-SC       X40        181292.0      2
M-SC       XR         178268.0      2
H-MC       AR         168868.0      2
M-LC       XR         145298.0      1
X-LC       XY25       128401.0      2
L-SC       OX40N       95173.0      1
           XR          92450.0      1
M-SC       AR          74543.0      1
H-LC       AR          70122.0      1
M-MC       XY25        50323.0      1
L-MC       XR          49554.0      1
M-MC       AR          47050.0      1
L-LC       XY25        20949.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 32.0 seconds
